# Starcraft II Zerg Unit Supply Formula - Genetic Algorithm

## Project Overview
This project uses a Genetic Algorithm (GA) to find a mathematical formula that approximates the supply cost of Zerg units in Starcraft II based on their internal Unit IDs. The goal is to replace a hardcoded lookup table with a compact formula.

## Problem Definition
- **Input**: Unit ID (integer `x`).
- **Output**: Supply cost (float/integer).
- **Objective**: Minimize the difference between the formula's output and the actual supply cost for all Zerg units.

In [13]:
zerg_units = {
    0:  ("Larva",       0),
    1:  ("Drone",       1),
    2:  ("Overlord",    -8),
    3:  ("Overseer",    -8),
    4:  ("Queen",       2),

    5:  ("Zergling",    0.5),
    6:  ("Baneling",    0.5),

    7:  ("Roach",       2),
    8:  ("Ravager",     3),

    9:  ("Hydralisk",   2),
    10: ("Lurker",      3),

    11: ("Mutalisk",    2),
    12: ("Corruptor",   2),
    13: ("Brood Lord",  4),

    14: ("Swarm Host",  3),
    15: ("Infestor",    2),
    16: ("Viper",       3),
    17: ("Ultralisk",   6),
}

## Methodology

### Chromosome Representation
Solutions are represented as strings forming mathematical expressions.
- **Alphabet**: `0123456789+-*/|&^%x`
- **Variable**: `x` represents the Unit ID.
- **Example**: `x/2+1`

### Fitness Function
The fitness function quantifies the accuracy of an expression.
$$ Fitness = 10000 - \sum_{units} (Predicted - Actual)^2 \times 25 - Penalties $$
Penalties are applied for:
- Invalid syntax (eval failure).
- Missing variable `x`.
- Lack of operators.
- Excessive length (slight penalty to encourage shorter formulas).

### Genetic Operators
- **Selection**: Tournament Selection (to maintain diversity).
- **Crossover**: Single-point crossover at operator positions to maintain valid syntax probability.
- **Mutation**: Random insertion, deletion, or substitution of characters.

In [28]:
%%capture --no-stdout
# Suppress warnings of eval

import random
from functools import lru_cache

MAX_LENGTH = 50
alphabet = "0123456789+-*/|&^%x"

def init_population(pop_size, max_length):
    population = []
    for _ in range(pop_size):
        # Start with simple expressions to bootstrap
        if random.random() < 0.5:
            individual = ''.join(random.choices(alphabet, k=random.randint(1, max_length)))
        else:
            # Try to generate something with x
            individual = 'x' + random.choice("+-*/%|&^") + random.choice("0123456789")
        population.append(individual)
    return population

def crossover(parent1, parent2):
    # Try to cut at operators to preserve validity
    op1_list = list("+-*/%|&^")
    op2_list = list("+-*/%|&^")
    random.shuffle(op1_list)
    random.shuffle(op2_list)
    
    for op1 in op1_list:
        cut1 = parent1.find(op1)
        if cut1 == -1: continue
        for op2 in op2_list:
            cut2 = parent2.rfind(op2)
            if cut2 == -1: continue
            
            child = parent1[:cut1] + parent2[cut2:]
            if len(child) > MAX_LENGTH:
                continue
            try:
                # Quick check if it compiles
                compile(child, '<string>', 'eval')
                return child
            except:
                continue
    
    # Fallback: simple concatenation with operator
    child = parent1 + random.choice("+-*/%|&^") + parent2
    if len(child) > MAX_LENGTH:
        child = child[:MAX_LENGTH]
    return child

def mutate(individual, mutation_rate, max_mutations=2):
    if random.random() > mutation_rate:
        return individual
        
    mutation_type = random.choice(['insertion', 'deletion', 'substitution'])
    pos = random.randint(0, len(individual) - 1)
    
    if mutation_type == 'insertion':
        if len(individual) < MAX_LENGTH:
            char = random.choice(alphabet)
            individual = individual[:pos] + char + individual[pos:]
    elif mutation_type == 'deletion' and len(individual) > 1:
        individual = individual[:pos] + individual[pos+1:]
    elif mutation_type == 'substitution':
        char = random.choice(alphabet)
        individual = individual[:pos] + char + individual[pos+1:]
        
    return individual

def evaluate_fitness(individual, unit_data):
    total_error = 0
    
    # Basic validity checks
    if 'x' not in individual:
        return 0
    if not any(c in "+-/*%|&^" for c in individual):
        return 0
        
    try:
        code = compile(individual, '<string>', 'eval')
    except:
        return 0

    for x, (_unit_name, actual_supply) in unit_data.items():
        try:
            # Safe eval with restricted globals
            predicted_supply = eval(code, {"__builtins__": {}}, {"x": x})
            if not isinstance(predicted_supply, (int, float)):
                return 0
            error = (predicted_supply - actual_supply) ** 2
            total_error += error * 25
        except:
            return 0

    # have tried to use "does contain x" as a penalty, but it got just x//4 for every variant
    # or in some cases it returned constants like 0x3 to avoid panelaties
    
    # if 'x' not in individual:
    #     total_error += 150  # Heavy penalty for not using x

    if not any(c in "+-/*%|&^" for c in individual):
        total_error += 100  # Penalty for lack of operators

    fitness = 5000 - total_error - len(individual)
    return max(0, fitness)

def tournament_selection(population, fitnesses, k=3):
    selected = random.sample(list(zip(population, fitnesses)), k)
    return max(selected, key=lambda x: x[1])[0]

def genetic_algorithm(unit_data, pop_size=50, generations=1000, mutation_rate=0.5, verbose=False):
    population = init_population(pop_size, MAX_LENGTH)
    best_overall_expr = None
    best_overall_fit = -float('inf')
    
    for gen in range(generations):
        fitnesses = [evaluate_fitness(ind, unit_data) for ind in population]
        
        # Track best
        max_fit = max(fitnesses)
        if max_fit > best_overall_fit:
            best_overall_fit = max_fit
            best_overall_expr = population[fitnesses.index(max_fit)]
            
        if verbose and gen % 100 == 0:
            print(f"Gen {gen}: Best Fitness = {max_fit:.2f}, Expr = {best_overall_expr}")
            
        # Elitism
        sorted_pop = sorted(zip(population, fitnesses), key=lambda x: x[1], reverse=True)
        new_population = [x[0] for x in sorted_pop[:2]] # Keep top 2
        
        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate)
            new_population.append(child)
            
        population = new_population
        
    return best_overall_expr, best_overall_fit


## Experiments
We run the GA with different configurations to observe the effect of population size and mutation rate.

In [29]:
import pandas as pd

configs = [
    {"pop_size": 50, "mutation_rate": 0.3},
    {"pop_size": 50, "mutation_rate": 0.5},
    {"pop_size": 50, "mutation_rate": 0.7},
    {"pop_size": 100, "mutation_rate": 0.3},
    {"pop_size": 100, "mutation_rate": 0.5},
    {"pop_size": 100, "mutation_rate": 0.7},
]

results = []

print("Running experiments...")
for config in configs:
    print(f"Running config: {config}")
    expr, fit = genetic_algorithm(zerg_units, 
                                pop_size=config["pop_size"], 
                                mutation_rate=config["mutation_rate"], 
                                generations=1000)
    results.append({
        "Population Size": config["pop_size"],
        "Mutation Rate": config["mutation_rate"],
        "Best Fitness": fit,
        "Best Expression": expr
    })

df = pd.DataFrame(results)
display(df)

Running experiments...
Running config: {'pop_size': 50, 'mutation_rate': 0.3}
Running config: {'pop_size': 50, 'mutation_rate': 0.5}
Running config: {'pop_size': 50, 'mutation_rate': 0.7}
Running config: {'pop_size': 100, 'mutation_rate': 0.3}
Running config: {'pop_size': 100, 'mutation_rate': 0.5}
Running config: {'pop_size': 100, 'mutation_rate': 0.7}


,Population Size,Mutation Rate,Best Fitness,Best Expression
0,50,0.3,1458.5,x//4
1,50,0.5,1458.5,x//4
2,50,0.7,1458.5,x//4
3,100,0.3,1458.5,x//4
4,100,0.5,1458.5,x//4
5,100,0.7,1458.5,x//4


## Verification
Verifying the best overall formula found.

In [30]:
best_row = df.loc[df['Best Fitness'].idxmax()]
best_expr = best_row['Best Expression']
print(f"Best Overall Expression: {best_expr}")

print("\nDetailed Check:")
for x, (name, actual) in zerg_units.items():
    try:
        pred = eval(best_expr, {"__builtins__": {}}, {"x": x})
        print(f"{name}: Actual={actual}, Predicted={pred:.2f}, Error={(pred-actual)**2:.2f}")
    except:
        print(f"{name}: Error evaluating")

Best Overall Expression: x//4

Detailed Check:
Larva: Actual=0, Predicted=0.00, Error=0.00
Drone: Actual=1, Predicted=0.00, Error=1.00
Overlord: Actual=-8, Predicted=0.00, Error=64.00
Overseer: Actual=-8, Predicted=0.00, Error=64.00
Queen: Actual=2, Predicted=1.00, Error=1.00
Zergling: Actual=0.5, Predicted=1.00, Error=0.25
Baneling: Actual=0.5, Predicted=1.00, Error=0.25
Roach: Actual=2, Predicted=1.00, Error=1.00
Ravager: Actual=3, Predicted=2.00, Error=1.00
Hydralisk: Actual=2, Predicted=2.00, Error=0.00
Lurker: Actual=3, Predicted=2.00, Error=1.00
Mutalisk: Actual=2, Predicted=2.00, Error=0.00
Corruptor: Actual=2, Predicted=3.00, Error=1.00
Brood Lord: Actual=4, Predicted=3.00, Error=1.00
Swarm Host: Actual=3, Predicted=3.00, Error=0.00
Infestor: Actual=2, Predicted=3.00, Error=1.00
Viper: Actual=3, Predicted=4.00, Error=1.00
Ultralisk: Actual=6, Predicted=4.00, Error=4.00
